# Prepare input CytoSPACE

In [2]:
global_path = './cell-cell-communication/' ## Path to the github downloaded repository

In [3]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

out_path = global_path + 'Data/Heart/Input_cytoSPACE/'


## Load data 

In [4]:
# Load raw data
adata_sp = sc.read(global_path+'Data/Heart/visium-OCT_adult-8reg-revision_raw.h5ad')
adata_sc = sc.read(global_path+'Data/Heart/RNA_adult-8reg_full_raw_cellstate-annotated.h5ad')

adata_sc.var_names = adata_sc.var['gene_name-new'].astype(str)
adata_sp.var_names = adata_sp.var['SYMBOL']

adata_sc.var_names_make_unique()
adata_sp.var_names_make_unique()

In [8]:
# Select the SAN or the AVN data
adata_sc = adata_sc[adata_sc.obs.region == 'SAN'] # Select the region
adata_sc = adata_sc[adata_sc.obs.donor == 'AH1'] # Select the donor
adata_sc = adata_sc[adata_sc.obs.cell_state != 'unclassified'] # Remove unclassified cells

adata_sp = adata_sp[adata_sp.obs.region == 'SAN'] # Select the region
adata_sp = adata_sp[adata_sp.obs.sangerID == 'HCAHeartST10659160'] # Select the spatial slide

## Save raw scRNA-seq counts as tsv file

In [9]:
counts = pd.DataFrame(adata_sc.X.toarray()).T
counts.columns = adata_sc.obs.index
counts.index = list(adata_sc.var.index)
counts.index.name='GENES'
counts.to_csv(out_path + 'SAN/GEX.tsv', sep = '\t')

## Add Celltype information

In [ ]:
cell_label = pd.DataFrame(adata_sc.obs.cell_state)
cell_label.columns = ['CellType']
cell_label.index.name='Cell IDs'
cell_label.to_csv(out_path + 'SAN/cell_label.tsv', sep = '\t')

# Save raw spatial counts

In [ ]:
counts_sp = pd.DataFrame(adata_sp.X.toarray()).T
counts_sp.columns = adata_sp.obs.index
counts_sp.index = list(adata_sp.var.index)
counts_sp.index.name='V1'
counts_sp.to_csv(out_path + 'SAN/GEX_sp.tsv', sep = '\t')

# Add coords

In [ ]:
coords = adata_sp.obs[['array_row', 'array_col']]
coords.columns = ['row', 'col']
coords.index.name='SpotID'
coords.to_csv(out_path + 'SAN/coords.tsv', sep = '\t')

# Run cytoSPACE using the terminal (bash)

In [ ]:
cytospace \
   -sp ./Data/Heart/Input_cytoSPACE/SAN/GEX.tsv \
   -ctp ./Data/Heart/Input_cytoSPACE/SAN/cell_label.tsv \
   -stp ./Data/Heart/Input_cytoSPACE/SAN/GEX_sp.tsv \
   -cp ./Data/Heart/Input_cytoSPACE/SAN/coords.tsv \
   -sam place_holders \
   -o ./Results/Heart/CytoSPACE/SAN/ \   

